In [1]:
import time
import logging
from IPython.display import Audio

from Freesound.model import *
from IDRnD.utils import *
from IDRnD.augmentations import *
from IDRnD.dataset import *

import numpy as np
import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, CyclicLR
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

from sklearn.model_selection import StratifiedKFold, train_test_split

%reload_ext autoreload
%autoreload 2
%matplotlib inline

seed_everything(0)
logging.basicConfig(level=logging.DEBUG, filename="logs/logs.log", filemode="w+")

In [3]:
X, y = get_train_data()

In [4]:
#cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#train_index, test_index = next(cv.split(X, y))
#X_train, X_val = X[train_index], X[test_index]
#y_train, y_val = y[train_index], y[test_index]

In [5]:
post_transform = transforms.Compose([
    #ToMellSpec(n_mels=331),
    librosa.power_to_db,
    #GetMFCC(),
    PadOrClip(150),
    #Normalize_predef(-24.42848536693399, 13.524654994769366),
    #Normalize(),
    ToTensor(),
    #transforms.ToTensor(),
])

In [5]:
from librosa.display import specshow

In [6]:
train_dataset[0][0].shape

NameError: name 'train_dataset' is not defined

In [7]:
train_dataset[0][0]

NameError: name 'train_dataset' is not defined

In [ ]:
specshow(train_dataset[6][0][0].numpy())

In [6]:
from IDRnD.resnet import resnet34

In [7]:
##train
hm = Train(model_path="models/kaggle2.pt")

train_dataset = MelDataset(X_train, y_train, "../data/files/raw_mels", post_transform)
#train_dataset_mixmatch = Dataset_Train_Mixmatch(train_dataset, post_transform=post_transform, pad_lenth=256)
valid_dataset = MelDataset(X_val, y_val, "../data/files/raw_mels_val", post_transform)

train_loader = DataLoader(train_dataset, batch_size=60, num_workers=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=60, num_workers=16, shuffle=False)

#model = Classifier(1).cuda()
model = resnet34(num_classes=1).cuda()
model_dst = torch.nn.DataParallel(model, device_ids=[0, 1]).cuda()

#criterion = nn.BCEWithLogitsLoss(weight=torch.Tensor([5])).cuda()
criterion = nn.BCEWithLogitsLoss().cuda()
optimizer = Adam(params=model.parameters(), lr=1e-4)

In [ ]:
hm.fit(train_loader, valid_loader, model_dst, criterion, optimizer, None, epoches=1000)

In [6]:
b, c = next(iter(train_loader))

In [7]:
k = model(b.cuda())

In [8]:
k.shape

torch.Size([60, 1])

In [11]:
c.shape

torch.Size([60, 1])

In [12]:
k.shape

torch.Size([60, 1])